In [1]:
from Utils.scrapping_file import Scraping_Class
from Utils.constants import *

import pandas as pd
import numpy as np

In [2]:
scraping_class = Scraping_Class()
price_steps = [0,50,100,150,200,250,300,500,700,999,1000,3000]

In [3]:
final_df = pd.DataFrame(columns=['name','SKU','price','department','sector'])


for department in list(DEPARTMENT_SECTOR.keys()):
    print(f'===========================================================|d|\nscaning department: {department}')

    for sector in (DEPARTMENT_SECTOR[department]):
        print(f'===============================================|s|\nscaning sector: {sector}')
        all_links = []
        
        for n in range(len(price_steps)):
            if n+1<len(price_steps):
                min_price = price_steps[n]
                max_price = price_steps[n+1]

            print(f'=========|p|\nscaning prices: {min_price} to {max_price}')

            url = scraping_class.URL_TEMPLATE(department=department, sector=sector, page_num=1, min_price=min_price, max_price=max_price)

            num_products = scraping_class.scan_num_products(url)
            pages_num = int(np.ceil(num_products/28))
            if pages_num > 50:
                pages_num=50

            if pages_num>0:    
                for page in range(pages_num):
                    page+=1
                    print(f'scaning page: {page} of {pages_num}')
                    url = scraping_class.URL_TEMPLATE(department=department, sector=sector, page_num=page, min_price=min_price, max_price=max_price)

                    products_name, products_price_final, products_sku = scraping_class.scan_products(url)
                    
                    temp_df = pd.DataFrame({'name':products_name,
                                            'SKU':products_sku,
                                            'price':products_price_final,
                                            'department':[department]*len(products_sku),
                                            'sector':[sector]*len(products_sku)})
                    final_df = pd.concat([final_df,temp_df]).reset_index(drop=True)

        


===========================================================|d|
scaning department: baby
===============================================|s|
scaning sector: alimentacao-do-bebe
=========|p|
scaning prices: 0 to 50
scaning page: 1 of 50


TypeError: scan_products() got an unexpected keyword argument 'range_price'

In [3]:
final_df

NameError: name 'final_df' is not defined

: 

: 

In [5]:
final_df.to_csv("ri_happy_8.csv",sep=';', encoding='cp1252')

In [6]:
df0 = pd.read_csv('ri_happy_0.csv',sep=',')
df1 = pd.read_csv('ri_happy_1.csv',sep=';',  encoding='cp1252')
df2 = pd.read_csv('ri_happy_2.csv',sep=';',  encoding='cp1252')
df3 = pd.read_csv('ri_happy_3.csv',sep=';',  encoding='cp1252')
df4 = pd.read_csv('ri_happy_4.csv',sep=';',  encoding='cp1252')
df5 = pd.read_csv('ri_happy_5.csv',sep=';',  encoding='cp1252')
df6 = pd.read_csv('ri_happy_6.csv',sep=';',  encoding='cp1252')
df7 = pd.read_csv('ri_happy_7.csv',sep=';',  encoding='cp1252')
df8 = pd.read_csv('ri_happy_8.csv',sep=';',  encoding='cp1252')

final_df = pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8],ignore_index=True).reset_index(drop=True)


In [9]:
final_df.dropna(subset='SKU')

,Unnamed: 0,name,SKU,price,department,sector
1,1,Kit 2 Copos Colors Bico De Silicone Disney Min...,1.002755e+08,39.90,baby,alimentacao-do-bebe
2,2,Kit 2 Copos Colors Bico De Silicone Disney Mic...,1.002346e+08,39.90,baby,alimentacao-do-bebe
3,3,Bico NUK - Advanced de Silicone - Todos os Líq...,1.002388e+09,18.90,baby,alimentacao-do-bebe
6,6,Mordedor - Centopeia Baby - Buba,1.001234e+08,27.53,baby,alimentacao-do-bebe
7,7,Mamadeira - My First Boy - 300 ml S2 - Tubarão...,1.002785e+08,32.90,baby,alimentacao-do-bebe
...,...,...,...,...,...,...
23069,3865,MIni Mochila Escolar Infantil DisneyParks Minn...,1.002632e+09,2642.54,produtos importados,livros-e-papelaria
23070,3866,Mochila Escolar Infantil Loungefly Disney Minn...,1.002640e+09,2009.35,produtos importados,livros-e-papelaria
23071,3867,Mochila Escolar Infantil Loungefly Disney Minn...,1.002640e+09,2063.70,produtos importados,livros-e-papelaria
23072,3868,Painel De Atividades Psicomotor Sensorial De P...,1.002385e+09,2139.90,produtos importados,livros-e-papelaria


In [12]:
final_df.drop('Unnamed: 0',axis=1).to_csv("ri_happy.csv",sep=';', encoding='cp1252')